In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import logging
from pathlib import Path
import numpy as np
from tqdm import tqdm
from train import Deep4SNet

In [2]:

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:

class DeepfakeDetector:
    def __init__(self, model_path):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")
        
        # Load model architecture
        self.model = Deep4SNet()
        
        # Load model weights
        checkpoint = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model = self.model.to(self.device)
        self.model.eval()
        
        logger.info(f"Model loaded from {model_path}")
        logger.info(f"Best validation accuracy: {checkpoint['val_acc']:.2f}%")
        
        # Define transform
        self.transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])
    
    def predict_single(self, image_path):
        """Predict for a single image"""
        try:
            # Load and transform image
            with Image.open(image_path) as img:
                if img.mode == 'RGBA':
                    img = img.convert('RGB')
                image = self.transform(img)
                image = image.unsqueeze(0)  # Add batch dimension
                
            # Predict
            with torch.no_grad():
                image = image.to(self.device)
                outputs = self.model(image)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                pred_prob, predicted = torch.max(probabilities, 1)
                
            # Get results
            is_fake = predicted.item() == 1
            confidence = pred_prob.item() * 100
            
            return {
                'prediction': 'FAKE' if is_fake else 'REAL',
                'confidence': confidence,
                'probabilities': {
                    'real': probabilities[0][0].item() * 100,
                    'fake': probabilities[0][1].item() * 100
                }
            }
            
        except Exception as e:
            logger.error(f"Error processing image {image_path}: {str(e)}")
            return None
    
    def predict_batch(self, image_folder, pattern="*.png"):
        """Predict for all images in a folder"""
        results = []
        image_paths = list(Path(image_folder).glob(pattern))
        
        if not image_paths:
            logger.warning(f"No images found in {image_folder} with pattern {pattern}")
            return results
        
        logger.info(f"Processing {len(image_paths)} images from {image_folder}")
        
        for image_path in tqdm(image_paths):
            result = self.predict_single(image_path)
            if result:
                results.append({
                    'path': str(image_path),
                    'prediction': result['prediction'],
                    'confidence': result['confidence'],
                    'probabilities': result['probabilities']
                })
        
        # Calculate statistics
        predictions = [r['prediction'] for r in results]
        confidences = [r['confidence'] for r in results]
        
        stats = {
            'total_images': len(results),
            'real_count': predictions.count('REAL'),
            'fake_count': predictions.count('FAKE'),
            'avg_confidence': np.mean(confidences),
            'min_confidence': np.min(confidences),
            'max_confidence': np.max(confidences)
        }
        
        return results, stats

In [30]:
import os

def main():
    # Initialize detector
    detector = DeepfakeDetector(r'D:\DeepLearning-Project (virtual-env)\DeepfakeDetection\models\best_model.pth')
    
    # Example 2: Batch prediction for both fake and real image sets
    fake_folder_path = r'D:\DeepLearning-Project (virtual-env)\DeepfakeDetection\Data\H-Voice_SiF-Filtered\Test_Set\fake'
    real_folder_path = r'D:\DeepLearning-Project (virtual-env)\DeepfakeDetection\Data\H-Voice_SiF-Filtered\Test_Set\real'
    
    # Check if the folder paths contain images
    print("Checking fake folder contents:", os.listdir(fake_folder_path))
    print("Checking real folder contents:", os.listdir(real_folder_path))
    
    try:
        # Perform batch prediction for fake images
        fake_results = detector.predict_batch(fake_folder_path, pattern="*.png")
        if not fake_results:
            print("No results found for fake images.")
            fake_results, fake_stats = [], {"total_images": 0, "fake_count": 0, "real_count": 0, "avg_confidence": 0}
        else:
            fake_results, fake_stats = fake_results

        # Perform batch prediction for real images
        real_results = detector.predict_batch(real_folder_path, pattern="*.png")
        if not real_results:
            print("No results found for real images.")
            real_results, real_stats = [], {"total_images": 0, "fake_count": 0, "real_count": 0, "avg_confidence": 0}
        else:
            real_results, real_stats = real_results

    except Exception as e:
        print(f"Error occurred during batch prediction: {e}")
        return  # Exit the function if there's an error
    
    # Calculate total statistics
    total_images = fake_stats['total_images'] + real_stats['total_images']
    fake_count = fake_stats['fake_count']
    real_count = real_stats['real_count']
    
    # Calculate average confidence for both real and fake images
    avg_confidence = (fake_stats['avg_confidence'] * fake_stats['total_images'] + 
                      real_stats['avg_confidence'] * real_stats['total_images']) / total_images
    
    # Calculate accuracy: (True Positives + True Negatives) / Total Images Processed
    true_positive = fake_count  # Fake images correctly detected as fake
    true_negative = real_count  # Real images correctly detected as real
    accuracy = (true_positive + true_negative) / total_images * 100 if total_images > 0 else 0
    
    # Calculate precision and recall
    false_positive = real_stats['fake_count']  # Real images incorrectly detected as fake
    false_negative = fake_stats['real_count']  # Fake images incorrectly detected as real
    
    precision = (true_positive / (true_positive + false_positive) * 100) if (true_positive + false_positive) > 0 else 0
    recall = (true_positive / (true_positive + false_negative) * 100) if (true_positive + false_negative) > 0 else 0

    # Print the results for fake and real batches
    print("\nBatch Processing Results for Fake Data:")
    print(f"Total fake images processed: {fake_stats['total_images']}")
    print(f"Fake images detected: {fake_stats['fake_count']}")
    print(f"Real images detected: {fake_stats['real_count']}")
    
    print("\nBatch Processing Results for Real Data:")
    print(f"Total real images processed: {real_stats['total_images']}")
    print(f"Real images detected: {real_stats['real_count']}")
    print(f"Fake images detected: {real_stats['fake_count']}")
    
    # Print combined statistics
    print("\nCombined Results (Fake + Real):")
    print(f"Total images processed: {total_images}")
    print(f"Real images detected: {real_count}")
    print(f"Fake images detected: {fake_count}")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.2f}%")
    print(f"Recall: {recall:.2f}%")
    
    # Save detailed results for both batches
    with open('detection_results.txt', 'w') as f:
        f.write("Detailed Detection Results (Fake and Real):\n\n")
        
        # Write results for fake images
        f.write("\nFake Images Results:\n")
        for result in fake_results:
            f.write(f"Image: {result['path']}\n")
            f.write(f"Prediction: {result['prediction']}\n")
            f.write(f"Real Probability: {result['probabilities']['real']:.2f}%\n")
            f.write(f"Fake Probability: {result['probabilities']['fake']:.2f}%\n")
            f.write("-" * 50 + "\n")
        
        # Write results for real images
        f.write("\nReal Images Results:\n")
        for result in real_results:
            f.write(f"Image: {result['path']}\n")
            f.write(f"Prediction: {result['prediction']}\n")
            f.write(f"Real Probability: {result['probabilities']['real']:.2f}%\n")
            f.write(f"Fake Probability: {result['probabilities']['fake']:.2f}%\n")
            f.write("-" * 50 + "\n")

if __name__ == "__main__":
    main()


INFO:__main__:Using device: cpu
C:\Users\anany\AppData\Local\Temp\ipykernel_38452\1816392652.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(mod

Checking fake folder contents: ['.DS_Store', 'fake (1).jpg', 'fake (10).jpg', 'fake (100).jpg', 'fake (101).jpg', 'fake (102).jpg', 'fake (103).jpg', 'fake (104).jpg', 'fake (105).jpg', 'fake (106).jpg', 'fake (107).jpg', 'fake (108).jpg', 'fake (109).jpg', 'fake (11).jpg', 'fake (110).jpg', 'fake (111).jpg', 'fake (112).jpg', 'fake (113).jpg', 'fake (114).jpg', 'fake (115).jpg', 'fake (116).jpg', 'fake (117).jpg', 'fake (118).jpg', 'fake (119).jpg', 'fake (12).jpg', 'fake (120).jpg', 'fake (121).jpg', 'fake (122).jpg', 'fake (123).jpg', 'fake (124).jpg', 'fake (125).jpg', 'fake (126).jpg', 'fake (127).jpg', 'fake (128).jpg', 'fake (129).jpg', 'fake (13).jpg', 'fake (130).jpg', 'fake (131).jpg', 'fake (132).jpg', 'fake (133).jpg', 'fake (134).jpg', 'fake (135).jpg', 'fake (136).jpg', 'fake (137).jpg', 'fake (138).jpg', 'fake (139).jpg', 'fake (14).jpg', 'fake (140).jpg', 'fake (141).jpg', 'fake (142).jpg', 'fake (143).jpg', 'fake (144).jpg', 'fake (145).jpg', 'fake (146).jpg', 'fake (1

100%|██████████| 758/758 [00:07<00:00, 96.89it/s] 
INFO:__main__:Processing 668 images from D:\DeepLearning-Project (virtual-env)\DeepfakeDetection\Data\H-Voice_SiF-Filtered\Test_Set\real
100%|██████████| 668/668 [00:06<00:00, 101.52it/s]


Batch Processing Results for Fake Data:
Total fake images processed: 758
Fake images detected: 654
Real images detected: 104

Batch Processing Results for Real Data:
Total real images processed: 668
Real images detected: 633
Fake images detected: 35

Combined Results (Fake + Real):
Total images processed: 1426
Real images detected: 633
Fake images detected: 654
Accuracy: 90.25%
Precision: 94.92%
Recall: 86.28%
